<a href="https://colab.research.google.com/github/scharnk/madam_marie/blob/main/BiddyTarot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Script to scrape all tarot card data from 'biddytarot.com' </h1>
<h4>Takes major and minor arcana card data down as dict() with card title as key and values including upright and reversed keywords, meanings, and overall card description.</h4>
<br>
78 card deck descriptions are based on Raider Waite imagery.<br><br>
<b>Minor card suits include: 

1. Cups
2. Swords
3. Pentacles
4. Wands
</b>


In [1]:
import seaborn as sns
sns.set()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
# import requests_cache
import dill
from bs4 import BeautifulSoup
from datetime import datetime
from retrying import retry
import re
import spacy
import itertools  # itertools.combinations may be useful
import networkx as nx

In [ ]:
page = requests.get('https://www.biddytarot.com/tarot-card-meanings/') # Use requests.get to download the page.

In [ ]:
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
url_base = 'https://www.biddytarot.com/'
url_end = ['/tarot-card-meanings/major-arcana/', '/tarot-card-meanings/minor-arcana/suit-of-cups/', '/tarot-card-meanings/minor-arcana/suit-of-swords/', '/tarot-card-meanings/minor-arcana/suit-of-pentacles/', '/tarot-card-meanings/minor-arcana/suit-of-wands/']

In [ ]:
soup.prettify

In [2]:
url_list = ['https://www.biddytarot.com/tarot-card-meanings/major-arcana/', 
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/']

In [5]:
page = requests.get(url_list[0]) # Use requests.get to download the page.
soup = BeautifulSoup(page.text, "html.parser")

In [7]:
url_major_arcana = [
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/fool/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/magician/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/high-priestess/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/empress/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/emperor/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hierophant/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/lovers/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/chariot/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/strength/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hermit/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/wheel-of-fortune/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/justice/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hanged-man/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/death/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/temperance/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/devil/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/tower/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/star/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/moon/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/sun/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/judgement/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/world/'
]

In [6]:
for link in soup.find_all('a'):
    print(link.get('href'))

#ajax-content-wrap
https://biddytarot.com/tarot-guides/tarot-card-answers/
#
https://www.biddytarot.com
#searchbox
#sidewidgetarea
/btc-free-training/
https://community.biddytarot.com/free-tarot-readings/
/learn-tarot
/learn-tarot
/tarot-card-meanings
/tarot-101/
/community
/community
http://community.biddytarot.com/login-page/
/blog
/category/podcast
/shop
/business
/about
/about
https://www.biddytarot.com/careers/
/contact-faq
https://www.biddytarot.com/diversity/
/login
#searchbox
/tarot-card-meanings/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://www.biddytarot.com/archetype-quiz/
/tarot-card-meanings/major-arcana/fool/
/tarot-card-meanings/major-arcana/magician/
/tarot-card-meanings/major-arcana/high-priestess/
/tarot-card-meanings/major-arcana/empress/
/tarot-card-meanings/major-arcana/emperor/
/t

In [ ]:
def get_links(response):
    soup = BeautifulSoup(response.text, "lxml")
    links = soup.select('div[a]')
    pairs = []
    for i in range(len(links)):
        pair = get_link_date(links[i])
        pairs.append(pair)
    return pairs # A list of URL, date pairs
    return links
    # for li in div:
    # all_li = li.find_all('li')
    # for link in all_li.find_all('a'):
    #     print(link.get('href'))

In [ ]:
link_list = []
def get_page():
    for i in range(1,26):
        page_url = "https://web.archive.org/web/20150918040703/http://www.newyorksocialdiary.com/party-pictures?page=" + str(i)
        response = requests.get(page_url)
        page_links = get_links(response)
        link_list.extend(page_links)
    return link_list

link_list = link_list[30:]

In [ ]:
link_list

[]

In [ ]:
# import re
# soup.find(string=re.compile("sisters"))

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [14]:
fool = requests.get('https://www.biddytarot.com/tarot-card-meanings/major-arcana/fool/')
soup_fool = BeautifulSoup(fool.text,"lxml")

In [34]:
# data_soup.find_all(attrs={"data-foo": "value"})

# soup_fool.find_all('span', attrs = {"class": "purple bold"})
soup_fool.find_all(attrs = {"class": "purple bold"})

SyntaxError: ignored

In [28]:
soup_fool.find_all('p')

[<p class="center bold padbot0 purple">The Right Tarot Card Meaning At The Right Time</p>,
 <p class="center purple bold fs32" style="padding-bottom: 25px;">Fool Tarot Card Meanings</p>,
 <p class="center"><span class="upper purple fs16"><strong>Love this deck?</strong></span><br/><a class="fs16" href="https://amzn.to/2pLU3KC" target="_blank">Buy the<br/>Everyday Tarot Deck</a></p>,
 <p><span class="purple bold">UPRIGHT:</span> Beginnings, innocence, spontaneity, a free spirit</p>,
 <p><span class="purple bold">REVERSED:</span> Holding back, recklessness, risk-taking</p>,
 <p>The Fool is numbered 0 – the number of unlimited potential – and so does not have a specific place in the sequence of the Tarot cards. The Fool can be placed either at the beginning of the Major Arcana or at the end. The Major Arcana is often considered the Fool’s journey through life and as such, he is ever present and therefore needs no number.</p>,
 <p>On the Fool Tarot card, a young man stands on the edge of a

In [27]:
soup_fool.find_all(id="reversed")

[<div class="container" id="reversed">
 <div class="row" style="padding: 50px 0; max-width: 900px; margin: 0 auto;">
 <div class="col span_8">
 <h2 class="center purple fs24 bold padbot15">Reversed Fool</h2>
 <div class="mobileshow padbot30">
 <img alt="Fool Tarot Card Meanings tarot card meaning" src="https://www.biddytarot.com/wp-content/uploads/2018/06/ET-MA-00-the-Fool.jpg" style="margin: 0 auto 15px; display: block; max-width: 275px; -webkit-transform: rotate(180deg); -moz-transform: rotate(180deg); -o-transform: rotate(180deg); -ms-transform: rotate(180deg); transform: rotate(180deg);" width="130"/>
 </div>
 <p>The Fool reversed suggests that you have conceived of a new project but aren’t ready to ‘birth’ it into the world just yet. You may worry that you are not fit or that you don’t have all the tools, skills and resources you need to make this project a success. Or perhaps you have a sense that the timing isn’t right. Something is holding you back, and you are preventing yours

In [19]:
soup_fool.get_text

<bound method Tag.get_text of <!DOCTYPE html>
<html class="no-js" lang="en-US">
<head> <script type="text/javascript">if(!gform){document.addEventListener("gform_main_scripts_loaded",function(){gform.scriptsLoaded=!0}),window.addEventListener("DOMContentLoaded",function(){gform.domLoaded=!0});var gform={domLoaded:!1,scriptsLoaded:!1,initializeOnLoaded:function(o){gform.domLoaded&&gform.scriptsLoaded?o():!gform.domLoaded&&gform.scriptsLoaded?window.addEventListener("DOMContentLoaded",o):document.addEventListener("gform_main_scripts_loaded",o)},hooks:{action:{},filter:{}},addAction:function(o,n,r,t){gform.addHook("action",o,n,r,t)},addFilter:function(o,n,r,t){gform.addHook("filter",o,n,r,t)},doAction:function(o){gform.doHook("action",o,arguments)},applyFilters:function(o){return gform.doHook("filter",o,arguments)},removeAction:function(o,n){gform.removeHook("action",o,n)},removeFilter:function(o,n,r){gform.removeHook("filter",o,n,r)},addHook:function(o,n,r,t,i){null==gform.hooks[o][n]&&(

In [8]:
major_page = requests.get('https://www.biddytarot.com/tarot-card-meanings/major-arcana/')

In [9]:
soup_major = BeautifulSoup(major_page.text, "lxml")

In [10]:
items = soup.findAll(True,{'class':'card_item'})

In [11]:
print(len(items))

22


In [13]:
print(items[0])

<div class="card_item">
<a href="/tarot-card-meanings/major-arcana/fool/">
<span class="card-item-wrap">
<img src="https://www.biddytarot.com/wp-content/uploads/2018/06/ET-MA-00-the-Fool.jpg"/>
<span class="card-item-content">
<h2 class="small purple upper center centered">Upright</h2>
<p>Beginnings, innocence, spontaneity, a free spirit</p>
<h2 class="small purple upper center centered">Reversed</h2>
<p>Holding back, recklessness, risk-taking</p>
<h4 class="gold centered center upper">Fool</h4>
</span>
</span>
</a>
</div>


In [ ]:
# for link in soup.find_all('a',href=True):
#     if pattern.match(link['href']):
#         print (link['href'])

for link in soup.findAll(True,{'class_':['card_item','a']}):
    print(link)

In [ ]:
from re import compile
# pattern = re.compile('major-arcana')

for link in items:
    # if pattern.match(link[href]):
        # print(link['href'])

    # print(link.find('a', href=True))


    # items = soup.findAll(True,{'class':'card_item'})

KeyError: ignored